In [ ]:
# This project detects face mask from images using Computer Vision.
# High-level implementation steps:
# 1. Load and preprocess the dataset
# 2. Build a CNN model using TensorFlow or PyTorch
# 3. Train the model on the dataset
# 4. Implement real-time detection using OpenCV

In [ ]:
# Install kaggle
!pip install kaggle

In [ ]:
# Creating and saving custom mask detection model
# Step 1: Dataset Acquisition

# Mount the Google drive so that you can store your Kaggle API credentials
from google.colab import drive
drive.mount('/content/drive')

# Create a Kaggle directory
!mkdir -p ~/.kaggle

# Copy the kaggle.json file to the Kaggle directory
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/

# Set permissions for the Kaggle API key
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset using the Kaggle API
!kaggle datasets download -d omkargurav/face-mask-dataset

# Unzip the dataset (if needed)
!unzip face-mask-dataset.zip

In [ ]:
# Step 2: Preprocess the dataset
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

# Define dataset directories
with_mask_dir = 'data/with_mask'
without_mask_dir = 'data/without_mask'

IMG_WIDTH = 128
IMG_HEIGHT = 128

def load_and_preprocess_images(directory, label):
    images = []
    labels = []
    for img_path in os.listdir(directory):
        try:
            img = cv2.imread(os.path.join(directory, img_path))
            img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])  # Resize the image

            # Check if the image was loaded successfully
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue

            img = img_to_array(img)
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue
    return images, labels

# Load images and labels
with_mask_images, with_mask_labels = load_and_preprocess_images(with_mask_dir, 1)
without_mask_images, without_mask_labels = load_and_preprocess_images(without_mask_dir, 0)

# Combine the data
images = with_mask_images + without_mask_images
labels = with_mask_labels + without_mask_labels

# Convert to numpy arrays and normalize
images = np.array(images, dtype="float32") / 255.0
labels = np.array(labels)

print("Number of images:", len(images))
print("Number of labels:", len(labels))
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

In [ ]:
# Step 3 : Build a CNN Model using Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# Assuming 'images' and 'labels' are your data and labels

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert to TensorFlow datasets
def create_dataset(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 64 # Since larger train_dataset(approx 6000 samples)
train_dataset = create_dataset(X_train, y_train, batch_size)
test_dataset = create_dataset(X_test, y_test, batch_size)

# Create the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=test_dataset
)

# Save the model
model.save('mask_detection_model.keras')

In [23]:
# Step 5: Implement detection using OpenCV from images
from google.colab.patches import cv2_imshow
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load the trained model
model = load_model('mask_detection_model.keras')

# Load the face detection cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_mask(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        face_roi = image[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))
        face_roi = img_to_array(face_roi)
        face_roi = preprocess_input(face_roi)
        face_roi = np.expand_dims(face_roi, axis=0)

        mask_prob = model.predict(face_roi)[0][0]

        if mask_prob > 0.5:
            label = f"Mask: {mask_prob:.2%}"
            color = (0, 255, 0)  # Green for mask
        else:
            no_mask_prob = 1 - mask_prob
            label = f"No Mask: {no_mask_prob:.2%}"
            color = (0, 0, 255)  # Red for no mask

        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

    return image

In [ ]:
# upload the image using google colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# Test on an image
test_image = cv2.imread('<your_image>')
result = detect_mask(test_image)
cv2_imshow(result)
cv2.waitKey(0)
cv2.destroyAllWindows()